In [ ]:
# -*- coding: utf-8 -*-
"""
Distribución de la población por edad (auto-actualizable desde World Bank)
Cambios:
  1) Anotación inferior derecha con fuente y fecha del dato más reciente.
  2) Barras por año: muestra los N años más recientes disponibles.
  3) Más espacio: mayor 'bargap', margen inferior y leyenda más abajo.
  4) Tabla de datos detallada añadida al final.
Fuentes:
  - Indicators API v2: https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-about-the-indicators-api-documentation
  - SP.POP.TOTL: https://data.worldbank.org/indicator/SP.POP.TOTL
  - SP.POP.0014.TO.ZS, SP.POP.1564.TO.ZS, SP.POP.65UP.TO.ZS
"""

import requests
import pandas as pd
import plotly.graph_objects as go
from datetime import date
import streamlit as st # Importación de Streamlit

# -----------------------------
# Parámetros
# -----------------------------
COUNTRY = "MEX"        # ISO3 (cámbialo si lo necesitas)
LAST_N_YEARS = 5       # cantidad de años recientes a graficar

INDICATORS = {
    "total": "SP.POP.TOTL",
    "pct_0_14": "SP.POP.0014.TO.ZS",
    "pct_15_64": "SP.POP.1564.TO.ZS",
    "pct_65_plus": "SP.POP.65UP.TO.ZS",
}
WB_BASE = "https://api.worldbank.org/v2/country/{country}/indicator/{indicator}?format=json&per_page=20000"

# -----------------------------
# Adaptación de Estilo para Streamlit
# -----------------------------
# Fallbacks con los colores originales si no hay inyección de Streamlit
DEFAULT_FONT = "Aptos Light, Aptos, Arial, sans-serif"
# Orden de colores: [0-14, 15-64, 65+]
DEFAULT_PALETTE = ["#0f1a24", "#889064", "#ff9f18"]

# Obtener variables inyectadas de Streamlit (si existen) o usar valores por defecto
PALETTE = globals().get('active_palette', DEFAULT_PALETTE)
FONT = globals().get('active_font', DEFAULT_FONT)

# Asignar variables de estilo que se usan en la gráfica
FONT_FAMILY = FONT
# Mapeamos la paleta inyectada a los colores específicos por grupo de edad
COL_0_14 = PALETTE[0] if len(PALETTE) > 0 else DEFAULT_PALETTE[0]     # azul oscuro
COL_15_64 = PALETTE[1] if len(PALETTE) > 1 else DEFAULT_PALETTE[1]    # verde corporativo
COL_65_PLUS = PALETTE[2] if len(PALETTE) > 2 else DEFAULT_PALETTE[2] # amarillo corporativo


# -----------------------------
# Utilidades
# -----------------------------
def fetch_indicator(country: str, indicator: str):
    """Descarga un indicador WB; regresa (df[year,value], last_updated_str)."""
    url = WB_BASE.format(country=country, indicator=indicator)
    resp = requests.get(url, timeout=60)
    resp.raise_for_status()
    js = resp.json()
    meta = js[0] if isinstance(js, list) and len(js) == 2 else {}
    last_updated = meta.get("lastupdated")  # p. ej. '2025-07-02'
    rows = js[1] if isinstance(js, list) and len(js) == 2 else []
    df = pd.DataFrame(rows)[["date", "value"]].dropna()
    df["year"] = df["date"].astype(int)
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    df = df.dropna(subset=["value"]).loc[:, ["year", "value"]].sort_values("year")
    return df.reset_index(drop=True), last_updated

def latest_common_year(dfs: list[pd.DataFrame]) -> int:
    common = set(dfs[0]["year"])
    for d in dfs[1:]:
        common &= set(d["year"])
    if not common:
        raise ValueError("No hay año en común entre los indicadores.")
    return max(common)

def to_millions(pop_total: float, pct: float) -> float:
    return round((pct / 100.0) * pop_total / 1e6, 1)

# -----------------------------
# Descarga
# -----------------------------
df_total, upd_total = fetch_indicator(COUNTRY, INDICATORS["total"])
df_0_14,  upd_0_14  = fetch_indicator(COUNTRY, INDICATORS["pct_0_14"])
df_15_64, upd_15_64 = fetch_indicator(COUNTRY, INDICATORS["pct_15_64"])
df_65p,   upd_65p   = fetch_indicator(COUNTRY, INDICATORS["pct_65_plus"])

# Fecha de actualización mostrada en el crédito (tomamos la más reciente entre las series)
updates = [d for d in [upd_total, upd_0_14, upd_15_64, upd_65p] if d]
last_update_str = max(updates) if updates else ""

# Años recientes comunes
common_years = sorted(set(df_total.year) & set(df_0_14.year) & set(df_15_64.year) & set(df_65p.year))
target_years = common_years[-LAST_N_YEARS:]  # N años más recientes

# Ensamble y cálculo (millones)
df = pd.DataFrame({"year": target_years}).merge(df_total.rename(columns={"value":"pop_total"}), on="year", how="left")
df = df.merge(df_0_14.rename(columns={"value":"pct_0_14"}),   on="year", how="left")
df = df.merge(df_15_64.rename(columns={"value":"pct_15_64"}), on="year", how="left")
df = df.merge(df_65p.rename(columns={"value":"pct_65_plus"}), on="year", how="left")

df["m_0_14"]   = df.apply(lambda r: to_millions(r["pop_total"], r["pct_0_14"]), axis=1)
df["m_15_64"]  = df.apply(lambda r: to_millions(r["pop_total"], r["pct_15_64"]), axis=1)
df["m_65_plus"]= df.apply(lambda r: to_millions(r["pop_total"], r["pct_65_plus"]), axis=1)

most_recent_year = target_years[-1]
hoy = date.today().isoformat()  # por si quieres mostrar "consultado el"

# -----------------------------
# Gráfica
# -----------------------------
fmt = lambda s: [str(v).replace(".", ",") for v in s]  # etiquetas con coma

fig = go.Figure()

fig.add_bar(
    name="0 - 14 años", x=df["year"], y=df["m_0_14"], marker_color=COL_0_14, # <-- Usa color dinámico
    text=fmt(df["m_0_14"]), textposition="inside", insidetextanchor="middle",
    textfont=dict(color="#dfe3e8", size=13),
    hovertemplate="Año: %{x}<br>0 - 14 años: %{y:.1f} millones<extra></extra>",
)
fig.add_bar(
    name="15 - 64 años", x=df["year"], y=df["m_15_64"], marker_color=COL_15_64, # <-- Usa color dinámico
    text=fmt(df["m_15_64"]), textposition="inside", insidetextanchor="middle",
    textfont=dict(color="#3e403f", size=13),
    hovertemplate="Año: %{x}<br>15 - 64 años: %{y:.1f} millones<extra></extra>",
)
fig.add_bar(
    name="65+ años", x=df["year"], y=df["m_65_plus"], marker_color=COL_65_PLUS, # <-- Usa color dinámico
    text=fmt(df["m_65_plus"]), textposition="inside", insidetextanchor="middle",
    textfont=dict(color="#3e403f", size=13),
    hovertemplate="Año: %{x}<br>65+ años: %{y:.1f} millones<extra></extra>",
)

# Layout con más espacio y crédito inferior derecho
fig.update_layout(
    barmode="stack",
    bargap=0.50,
    plot_bgcolor="white",
    paper_bgcolor="white",
    margin=dict(l=60, r=40, t=90, b=130),
    font=dict(family=FONT_FAMILY, size=18, color="#5f6368"), # <-- Usa fuente dinámica
    title=dict(
        text="Distribución población por edad<br><span style='font-size:13px;color:#9aa0a6'>(Millones)</span>",
        x=0.5, xanchor="center"
    ),
    xaxis=dict(
        title="",
        tickmode="array",
        tickvals=df["year"],
        ticktext=[str(y) for y in df["year"]],
        showgrid=False,
        tickfont=dict(color="#8a8f94", size=16)
    ),
    yaxis=dict(
        title="",
        showgrid=True,
        gridcolor="#e6e6e6",
        zeroline=False,
        rangemode="tozero",
        dtick=20,
        tickfont=dict(color="#8a8f94", size=16)
    ),
    legend=dict(
        title="",
        orientation="h",
        yanchor="bottom",
        y=-0.28,
        xanchor="center",
        x=0.5,
        font=dict(size=14)
    )
)

# --- Crédito inferior derecho (fuente + fecha del dato más reciente)
credito = (
    f"Fuente: World Bank – WDI (ONU-WPP). "
    f"Último año disponible: {most_recent_year}"
    + (f" · Actualización WDI: {last_update_str}" if last_update_str else "")
)
fig.add_annotation(
    x=0.6, y=-0.28, xref="paper", yref="paper",
    text=credito,
    showarrow=False,
    xanchor="right", yanchor="top",
    font=dict(family=FONT_FAMILY, size=12, color="#7a7a7a") # <-- Usa fuente dinámica
)

# -----------------------------
# Visualización en Streamlit
# -----------------------------
st.plotly_chart(fig, use_container_width=True)

# -----------------------------
# Tabla de Datos (Visible)
# -----------------------------
st.markdown("**Datos detallados (Millones)**")

# Preparamos el DataFrame para la tabla:
# Seleccionamos las columnas calculadas en millones y el año.
# Ordenamos descendente para ver lo más reciente arriba.
df_table = df[['year', 'm_0_14', 'm_15_64', 'm_65_plus']].copy().sort_values('year', ascending=False)

st.dataframe(
    df_table,
    use_container_width=True,
    hide_index=True,
    column_config={
        "year": st.column_config.TextColumn("Año"),
        "m_0_14": st.column_config.NumberColumn(
            "0 - 14 años",
            format="%.1f M"
        ),
        "m_15_64": st.column_config.NumberColumn(
            "15 - 64 años",
            format="%.1f M"
        ),
        "m_65_plus": st.column_config.NumberColumn(
            "65+ años",
            format="%.1f M"
        )
    }
)